In [1]:
!pip install peft

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `ssumday24` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `ssumday2

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

In [4]:
#연합학습 결과물인 어댑터 모델은 미리 허깅페이스에 업로드해놓아야 한다
peft_python_code_id = "prayslaks/python_code_instructions_18k_alpaca_20000_fedavg_c4s2_i10_b4a1_l512_r32a64_20241030071530"
peft_alpaca_cleaned_id = "prayslaks/alpaca-cleaned_20000_fedavg_c4s2_i10_b4a1_l512_r32a64_20241111092921"

adapter_id = peft_alpaca_cleaned_id

In [5]:
#Peft 어댑터의 설정 획득
config = PeftConfig.from_pretrained(adapter_id)
print(config.base_model_name_or_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

prometheus-eval/prometheus-7b-v2.0


In [6]:
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/789M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [8]:
model = PeftModel.from_pretrained(model, adapter_id)

adapter_model.safetensors:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
text = "Give three tips for staying healthy."
inputs = tokenizer(text, return_tensors="pt")

In [10]:
outputs = model.generate(**inputs, max_length=500, num_return_sequences=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Give three tips for staying healthy.

1. Eat a balanced diet: Eating a variety of nutrient-dense foods from all the food groups is essential for maintaining good health. Aim to include plenty of fruits, vegetables, whole grains, lean proteins, and healthy fats in your diet.

2. Stay active: Regular physical activity is important for overall health and well-being. Aim for at least 150 minutes of moderate aerobic activity or 75 minutes of vigorous aerobic activity each week, along with muscle-strengthening exercises on two or more days a week.

3. Get enough sleep: Adequate sleep is essential for good health. Aim for 7-9 hours of sleep each night, and establish a regular sleep schedule to help improve the quality of your sleep. Avoid screens and other stimulating devices before bedtime, and create a comfortable sleep environment to help you get the rest you need.


# 드라이브에 Combined 모델 저장

In [11]:
from google.colab import drive
drive.mount('/content/drive')

# 모델 저장
model.save_pretrained("/content/drive/MyDrive/peft_combined_model")
tokenizer.save_pretrained("/content/drive/MyDrive/peft_combined_model")


Mounted at /content/drive


('/content/drive/MyDrive/peft_combined_model/tokenizer_config.json',
 '/content/drive/MyDrive/peft_combined_model/special_tokens_map.json',
 '/content/drive/MyDrive/peft_combined_model/tokenizer.model',
 '/content/drive/MyDrive/peft_combined_model/added_tokens.json',
 '/content/drive/MyDrive/peft_combined_model/tokenizer.json')